In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:/Users/481536/Desktop/ML/diabetic/train.csv')
df_test = pd.read_csv(r'C:/Users/481536/Desktop/ML/diabetic/test.csv')

df.replace('?',np.nan,inplace=True)
df_test.replace('?',np.nan,inplace=True)

df.shape

(14696, 50)

In [ ]:
df.columns

Index(['encounter_id', 'patient_id', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'tel_1', 'tel_2', 'tel_3', 'tel_4', 'tel_5',
       'tel_6', 'tel_7', 'tel_8', 'tel_9', 'tel_10', 'tel_11', 'tel_12',
       'tel_13', 'tel_14', 'tel_15', 'tel_16', 'tel_17', 'tel_18', 'tel_19',
       'tel_20', 'tel_21', 'tel_22', 'tel_23', 'tel_24', 'tel_25', 'tel_26',
       'tel_27', 'tel_28', 'tel_29', 'tel_30', 'tel_41', 'tel_42', 'tel_43',
       'tel_44', 'tel_45', 'tel_46', 'tel_47', 'tel_48', 'tel_49',
       'diabetesMed'],
      dtype='object')

In [ ]:
df.isnull().any

In [ ]:
#Columns having missing value
cols=df.columns
cols_lst=[]
count_lst=[]
for col in cols:
    cnt=df[col][df[col] == '?'].count()
    if cnt > 0:
        cols_lst.append(col)
        count_lst.append(cnt)
df_nulls=pd.DataFrame({'Columns':cols_lst,'Count Of Nulls':count_lst})
df_nulls.sort_values(by='Count Of Nulls',ascending=False,inplace=True)
df_nulls=df_nulls.style.set_properties(**{'text-align': 'left'}).set_table_styles([ 
    dict(selector='th', props=[('text-align', 'left')] ) ])
df_nulls

C:\Users\481536\Anaconda3n\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Columns,Count Of Nulls
1,weight,14315
3,tel_2,8122
2,tel_1,2458
0,race,337
6,tel_11,139
5,tel_10,33
4,tel_9,1


In [ ]:
#Count of unique values in each column
r,c = df.shape
cols_lst=[]
count_lst=[]
cols=df.columns
print('Total rows in dataset:',r)
for col in cols:
    cols_lst.append(col)
    count_lst.append(len(df[col].unique()))
df_unique_values=pd.DataFrame({'Columns':cols_lst,'Count Of Unique values':count_lst})
df_unique_values.sort_values(by='Count Of Unique values',inplace=True, ascending=False)
df_unique_values=df_unique_values.style.set_properties(**{'text-align': 'left'}).set_table_styles([ 
    dict(selector='th', props=[('text-align', 'left')] ) ])
df_unique_values


Total rows in dataset: 14696


,Columns,Count Of Unique values
0,encounter_id,14696
1,patient_id,13011
20,tel_11,537
18,tel_9,512
19,tel_10,505
12,tel_3,104
14,tel_5,70
11,tel_2,48
15,tel_6,26
16,tel_7,22


In [ ]:
#List of unique values in each column
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None
cols_lst=[]
unique_vals_lst=[]
cols=df.columns
print('Total rows in dataset:',r)
for col in cols:
    unique_values=df[col].unique()
    cols_lst.append(col)
    if len(unique_values) < 100:
        unique_vals_lst.append(unique_values)
    else:
        unique_vals_lst.append('More than 100 unique values')
df_unique_values=pd.DataFrame({'Columns':cols_lst,'Unique values':unique_vals_lst})
df_unique_values=df_unique_values.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])

df_unique_values

Total rows in dataset: 14696


,Columns,Unique values
0,encounter_id,More than 100 unique values
1,patient_id,More than 100 unique values
2,race,['Caucasian' 'AfricanAmerican' 'Asian' '?' 'Hispanic' 'Other']
3,gender,['Male' 'Female']
4,age,['[80-90)' '[50-60)' '[70-80)' '[30-40)' '[60-70)' '[40-50)' '[10-20)' '[20-30)' '[90-100)' '[0-10)']
5,weight,['?' '[25-50)' '[50-75)' '[75-100)' '[125-150)' '[100-125)' '[150-175)' '[0-25)' '[175-200)']
6,admission_type_id,[1 3 2 5 8 6 4 7]
7,discharge_disposition_id,[ 3 1 23 6 5 2 13 22 7 4 11 14 18 28 24 15 8 9 25 19]
8,admission_source_id,[ 7 1 5 4 6 17 2 10 3 14 22 9 11 8 25 20]
9,time_in_hospital,[ 6 2 5 3 8 1 9 4 13 10 11 7 14 12]


In [ ]:
df.age.value_counts()

[70-80)     3828
[60-70)     3329
[80-90)     2605
[50-60)     2373
[40-50)     1326
[30-40)      486
[90-100)     445
[20-30)      228
[10-20)       63
[0-10)        13
Name: age, dtype: int64

In [ ]:
a,b,*_=df.age.value_counts()
print(a,b)
len(df.age.unique())

3828 3329


10

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

def fill_Missing_Values(df,field):
    df[field]=df[field].replace('?',np.nan)
    df[field].fillna(df[field].mode()[0],inplace = True)
    return df[field]

def clean_Using_Regex(df,regex,field):
    return [re.sub(regex,'',str(x)) for x in df[field]]

def clean_Data(df):
    #Fill missing values with Mode
    df['race']=fill_Missing_Values(df,'race')
    df['tel_10']=fill_Missing_Values(df,'tel_10')
    df['tel_11']=fill_Missing_Values(df,'tel_11')

    df.replace('?',0,inplace=True)
    
    #Dropping columns having lot of missing values
    df = df.drop(['weight','tel_1','tel_2'],axis=1)

    keys=['tel_15','tel_16','tel_17','tel_18','tel_19','tel_20','tel_21','tel_22','tel_23','tel_24','tel_25','tel_26','tel_27','tel_28','tel_29','tel_30','tel_41','tel_42','tel_43','tel_44','tel_45','tel_46','tel_47']
    for col in keys:
        df[col] = df[col].replace(['No','Steady','Up','Down'], [0,1,2,3])

    df = pd.get_dummies(df, columns=['race', 'gender','tel_13', 'tel_14', 'tel_48', 'tel_49'])

    df['age'] = [re.findall('^.([0-9]+)',x)[0] for x in df['age']]
    df['tel_9'] = clean_Using_Regex(df,'[^0-9.]','tel_9')
    df['tel_10'] = clean_Using_Regex(df,'[^0-9.]','tel_10')
    df['tel_11'] = clean_Using_Regex(df,'[^0-9.]','tel_11')

    encounter_id=df.encounter_id
	df.drop(['encounter_id','patient_id','admission_type_id','discharge_disposition_id','admission_source_id'],axis=1,inplace=True)

    return df,encounter_id

df,df_encounter_id=clean_Data(df)
df_test,df_test_encounter_id=clean_Data(df_test)

X_train=df.drop('diabetesMed',axis=1)
y_train=df['diabetesMed']
X_test=df_test

m=DecisionTreeClassifier()
m.fit(X_train,y_train)
y_pred=m.predict(X_test)

y_pred=pd.Series(y_pred)

output = {'encounter_id': df_test_encounter_id,'diabetesMed': y_pred}
submission = pd.DataFrame(output)
submission[['encounter_id','diabetesMed']].to_csv('submission.csv',index = False)